In [1]:
import pandas as pd
import numpy as np

In [2]:
train_df = pd.read_csv("3M_General_Target_data.csv")

/var/folders/3p/8hc9c44s38j902ytq253kt6m0000gn/T/ipykernel_1582/2682145492.py:1: DtypeWarning: Columns (254,260,339) have mixed types. Specify dtype option on import or set low_memory=False.
  train_df = pd.read_csv("3M_General_Target_data.csv")


In [3]:
# 독립변수(X) 후보군 추출 (식별자 및 타겟 변수 제외)
features_list = train_df.select_dtypes(include=[np.number]).drop(columns=['발급회원번호', '기준년월', '이탈_타겟'], errors='ignore').columns

# 전체 데이터(Full) 정제: 무한대 값 처리 및 결측치 0 채움
X_full = train_df[features_list].replace([np.inf, -np.inf], np.nan).fillna(0)
y_full = train_df['이탈_타겟']

# 모든 행의 값이 동일하여 변별력이 없는 컬럼은 사전에 제거
X_full = X_full.loc[:, (X_full != X_full.iloc[0]).any()]

In [4]:
from xgboost import XGBClassifier

# XGBoost 모델 생성 및 전체 데이터 학습 수행
# 모델은 전체 데이터의 복잡한 패턴을 모두 학습함
model = XGBClassifier(random_state=42, use_label_encoder=False, eval_metric='logloss')
model.fit(X_full, y_full)

/usr/local/Caskroom/miniconda/base/envs/myenv/lib/python3.13/site-packages/xgboost/training.py:199: UserWarning: [11:21:17] WARNING: /Users/runner/miniforge3/conda-bld/xgboost-split_1764148644238/work/src/learner.cc:790: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


,objective,'binary:logistic'
,base_score,None
,booster,None
,callbacks,None
,colsample_bylevel,None
,colsample_bynode,None
,colsample_bytree,None
,device,None
,early_stopping_rounds,None
,enable_categorical,False
,eval_metric,'logloss'


In [5]:
from statsmodels.stats.outliers_influence import variance_inflation_factor

# 1. 분석용 균형 샘플링 수행 (타겟 0과 1을 각각 최대 600개씩 추출)
# 이탈자(1)와 유지자(0)의 특성을 균형 있게 반영하기 위함
n_samples = 2000
df_0 = train_df[train_df['이탈_타겟'] == 0]
df_1 = train_df[train_df['이탈_타겟'] == 1]

sample_0 = df_0.sample(n=min(len(df_0), n_samples), random_state=42)
sample_1 = df_1.sample(n=min(len(df_1), n_samples), random_state=42)
sample_df = pd.concat([sample_0, sample_1])

# 2. 샘플 데이터 독립변수(X) 정제
# 전체 모델에서 사용된 컬럼과 동일한 구조로 구성
X_sample = sample_df[X_full.columns].replace([np.inf, -np.inf], np.nan).fillna(0)

# 3. VIF(Variance Inflation Factor) 지수 산출 (샘플 데이터 기준)
vif_data = pd.DataFrame()
vif_data["Column"] = X_sample.columns
vif_data["VIF"] = [variance_inflation_factor(X_sample.values, i) for i in range(X_sample.shape[1])]

/usr/local/Caskroom/miniconda/base/envs/myenv/lib/python3.13/site-packages/statsmodels/stats/outliers_influence.py:197: RuntimeWarning: divide by zero encountered in scalar divide
  vif = 1. / (1. - r_squared_i)
/usr/local/Caskroom/miniconda/base/envs/myenv/lib/python3.13/site-packages/statsmodels/regression/linear_model.py:1782: RuntimeWarning: invalid value encountered in scalar divide
  return 1 - self.ssr/self.centered_tss
/usr/local/Caskroom/miniconda/base/envs/myenv/lib/python3.13/site-packages/statsmodels/regression/linear_model.py:1782: RuntimeWarning: divide by zero encountered in scalar divide
  return 1 - self.ssr/self.centered_tss


In [6]:
import shap

# 4. SHAP Value 계산 (샘플 데이터 기준)
# 이미 학습된 전체 데이터 모델을 활용하여 샘플 데이터의 변수 기여도를 분석함
explainer = shap.TreeExplainer(model)
shap_values = explainer.shap_values(X_sample)

# 5. 변수별 평균 절대 SHAP 지수 산출
shap_importance = pd.DataFrame({
    'Column': X_sample.columns,
    'SHAP': np.abs(shap_values).mean(axis=0)
})

# 6. 최종 결과 통합 및 정렬 (컬럼, SHAP, VIF)
final_analysis = pd.merge(shap_importance, vif_data, on='Column')
final_analysis = final_analysis.sort_values(by='SHAP', ascending=False).reset_index(drop=True)

# 7. 최종 결과 출력
print(f"분석 샘플 수: 타겟0({len(sample_0)}건), 타겟1({len(sample_1)}건)")
print(final_analysis)

/usr/local/Caskroom/miniconda/base/envs/myenv/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


분석 샘플 수: 타겟0(2000건), 타겟1(2000건)
               Column       SHAP         VIF
0           당월_총_이용금액  10.500513         inf
1       직전_3M_평균_이용금액   1.233414         inf
2         이용금액_신용_B0M   1.018651         inf
3        이용금액_일시불_B0M   0.860215         inf
4         이용금액_신판_B0M   0.395204         inf
..                ...        ...         ...
691      이용건수_할부_R12M   0.000000         inf
692          카드이용한도금액   0.000000   26.409145
693  이용건수_할부_무이자_R12M   0.000000         inf
694            RV신청일자   0.000000  538.482938
695      이용개월수_전체_R6M   0.000000   64.506104

[696 rows x 3 columns]


In [ ]:
# file_path = 'General_Shap_Vif.csv'
# final_analysis.to_csv(file_path, index=False, encoding='utf-8-sig')

# # 파일 저장 완료 확인 메시지 출력
# print(f"최종 분석 결과가 다음 경로에 저장되었습니다: {file_path}")

In [7]:
features_list = [
    '당월_총_이용금액', '직전_3M_평균_이용금액', '이용금액_신용_B0M', '이용금액_일시불_B0M', '이용금액_신판_B0M',
    '이용금액_신용_R3M', '상향가능CA한도금액', 'CA이자율_할인전', '월상환론한도금액', '이용금액_일시불_R3M',
    '이용금액_R3M_신용', '증감율_이용금액_일시불_전월', '이용금액_일시불_R6M', '정상입금원금_B2M', '이용금액_할부_R3M',
    '변동률_잔액_B1M', '변동률_일시불평잔', '불만제기후경과월_R12M', '이용건수_오프라인_B0M', '증감율_이용금액_신판_전월',
    '정상청구원금_B0M', '대표결제일', 'CL이자율_할인전', '이용건수_신용_R3M', '최종유효년월_신용_이용',
    '입회경과개월수_신용', '월중평잔_일시불', '정상입금원금_B5M', '변동률_잔액_일시불_B1M', '연체입금원금_B2M',
    '최종이용일자_일시불', '정상청구원금_B2M', '보유여부_해외겸용_본인', 'CA한도금액', '잔액_신판최대한도소진율_r3m',
    '포인트_이용포인트_R12M', '연속유실적개월수_기본_24M_카드', 'RP금액_B0M', '카드이용한도금액_B1M',
    '증감율_이용금액_할부_전월', '증감율_이용금액_신판_분기', '최대이용금액_신용_R12M', '이용개월수_오프라인_R6M',
    '수신거부여부_TM', '변동률_CA평잔', '이용금액_오프라인_R6M', '납부_기타이용금액', '잔액_신판ca평균한도소진율_r3m',
    '증감율_이용금액_신용_전월', '이용건수_신판_R6M', '이용금액_R3M_신용체크', '이용금액_부분무이자_R3M',
    '이용금액_체크_R12M', '평잔_6M', '증감율_이용금액_일시불_분기', '_1순위쇼핑업종_이용금액', '이용금액_신판_R6M',
    '_1순위카드이용건수', '최대이용금액_체크_R12M', '이용건수_오프라인_R3M', '최종카드발급일자', '잔액_신판ca최대한도소진율_r3m',
    '이용금액_쇼핑', '월중평잔_일시불_B0M', '평잔_일시불_6M', '이용여부_3M_해외겸용_본인', '정상입금원금_B0M',
    '이용건수_일시불_R12M', '이용금액_신판_R3M', '이용금액_오프라인_R3M', '잔액_신판평균한도소진율_r3m',
    'RV일시불이자율_할인전', '_1순위카드이용금액', '이용금액_신용_R6M', '_1순위업종_이용금액', '이용금액_오프라인_B0M',
    '증감율_이용건수_할부_전월', '최종카드발급경과월', '수신거부여부_메일', '할인금액_R3M', 'OS구분코드', '최종탈회후경과월',
    '이용가맹점수', '상향가능한도금액', '인입월수_ARS_R6M', '쇼핑_도소매_이용금액', '최초한도금액', '혜택수혜율_R3M',
    'RV현금서비스이자율_할인전', '연체입금원금_B5M', '잔액_할부_무이자_B0M', '혜택수혜금액_R3M', '이용건수_신용_R6M',
    '이용금액_업종기준', '이용건수_신용_R12M', '증감율_이용금액_신용_분기', '증감율_이용건수_일시불_분기',
    '이용금액_신판_R12M', '_2순위카드이용건수', '혜택수혜율_B0M'
]

In [ ]:
# # SHAP 지수 기준 상위 100개 변수 추출
# # 분석 결과 데이터프레임(final_analysis)은 이미 SHAP 기준 내림차순 정렬되어 있음
# top_100_features = final_analysis.head(100)['Column'].tolist()

# # 추출된 변수 개수 확인
# print(f"선정된 핵심 변수 개수: {len(top_100_features)}개")

# # (선택) 상위 10개 변수 명단 확인
# print("--- [상위 10개 핵심 변수] ---")
# print(top_100_features[:10])

In [9]:
# 1. 분석 및 학습에 필수적인 기본 컬럼 정의
base_cols = ['발급회원번호', '기준년월', '이탈_타겟']

# 2. 앞서 정의한 features_list와 기본 컬럼 합치기
# 중복 선택을 방지하기 위해 set을 사용하거나 리스트를 더합니다.
selected_columns = base_cols + [col for col in features_list if col not in base_cols]

# 3. 데이터프레임 생성
# train_df에서 해당 컬럼들만 추출 (원본 보호를 위해 .copy() 사용)
final_analysis_df = train_df[selected_columns].copy()

# 4. 결과 확인
print(f"최종 추출된 데이터 크기: {final_analysis_df.shape}")
print(f"포함된 컬럼 수: {len(final_analysis_df.columns)}개")

# 상위 5개 데이터 확인
display(final_analysis_df.head())

최종 추출된 데이터 크기: (174339, 103)
포함된 컬럼 수: 103개


,발급회원번호,기준년월,이탈_타겟,당월_총_이용금액,직전_3M_평균_이용금액,이용금액_신용_B0M,이용금액_일시불_B0M,이용금액_신판_B0M,이용금액_신용_R3M,상향가능CA한도금액,...,잔액_할부_무이자_B0M,혜택수혜금액_R3M,이용건수_신용_R6M,이용금액_업종기준,이용건수_신용_R12M,증감율_이용금액_신용_분기,증감율_이용건수_일시불_분기,이용금액_신판_R12M,_2순위카드이용건수,혜택수혜율_B0M
0,SYN_1000048,201807,0,640723,550060.666667,640723,640723,640723,1650182,235,...,0,0,159,1697361,332,-0.420396,0.058658,9537395,45,0.0
1,SYN_1000048,201808,0,553131,467608.666667,553131,553131,553131,1402826,225,...,0,0,166,1874291,331,-0.377131,-0.200396,8368025,40,0.0
2,SYN_1000048,201809,0,593691,595848.333333,593691,593691,593691,1787545,239,...,0,0,153,1992527,321,0.121721,0.134754,7774755,37,0.0
3,SYN_1000048,201810,0,548379,565067.000000,548379,548379,548379,1695201,228,...,0,0,165,1967486,368,0.019740,0.385747,9856513,44,0.0
4,SYN_1000048,201811,0,592141,578070.333333,592141,592141,592141,1734211,244,...,0,0,159,1904224,384,0.624895,0.553486,7978739,51,0.0


In [10]:
target_logic_columns = [
    '이용금액_신용_B0M',
    '이용금액_체크_B0M',
    '이용금액_신용_R3M',
    '이용금액_체크_R3M',
    '당월_총_이용금액',
    '직전_3M_평균_이용금액'
]

In [11]:
# 1. 학습에서 제외할 '리키지(Leakage)' 변수 리스트
# 정답을 정의할 때 썼던 6개 변수 + 식별자들입니다.
leakage_cols = [
    '이용금액_신용_B0M', '이용금액_체크_B0M', '이용금액_신용_R3M', '이용금액_체크_R3M',
    '당월_총_이용금액', '직전_3M_평균_이용금액',
    '발급회원번호', '기준년월', '이탈_타겟'
]

# 2. 독립변수(X)와 종속변수(y) 분리
# selected_columns 중 위 리스트에 없는 것들만 X로 사용합니다.
X = final_analysis_df.drop(columns=leakage_cols, errors='ignore')
y = final_analysis_df['이탈_타겟']

print(f"최종 학습에 사용될 변수 개수: {len(X.columns)}개")

최종 학습에 사용될 변수 개수: 96개


In [12]:
from sklearn.model_selection import train_test_split
from xgboost import XGBClassifier
from sklearn.metrics import classification_report, roc_auc_score

# 1. 학습 데이터와 테스트 데이터 분할 (8:2)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# 2. 모델 생성 및 학습
model = XGBClassifier(n_estimators=100, learning_rate=0.1, max_depth=5, random_state=42)
model.fit(X_train, y_train)

# 3. 예측 및 평가
y_pred = model.predict(X_test)
y_prob = model.predict_proba(X_test)[:, 1]

print("--- [모델 평가 결과] ---")
print(classification_report(y_test, y_pred))
print(f"AUC Score: {roc_auc_score(y_test, y_prob):.4f}")

--- [모델 평가 결과] ---
              precision    recall  f1-score   support

           0       1.00      1.00      1.00     32176
           1       0.98      0.97      0.97      2692

    accuracy                           1.00     34868
   macro avg       0.99      0.98      0.99     34868
weighted avg       1.00      1.00      1.00     34868

AUC Score: 0.9997


In [13]:
# 1. 시간순 분할 (12월을 시험지로 사용)
last_month = final_analysis_df['기준년월'].max()
train_df_time = final_analysis_df[final_analysis_df['기준년월'] < last_month]
test_df_time = final_analysis_df[final_analysis_df['기준년월'] == last_month]

# 2. 제거할 '미래/정답' 관련 변수 리스트
# 타겟을 만들 때 쓴 6개 + B0M(당월)이 들어간 모든 변수 + 식별자
drop_cols = [col for col in final_analysis_df.columns if '_B0M' in col or '당월' in col] + \
            ['이용금액_신용_R3M', '이용금액_체크_R3M', '직전_3M_평균_이용금액', '발급회원번호', '기준년월', '이탈_타겟']

# 3. X(문제집), y(정답지) 분리
X_train = train_df_time.drop(columns=drop_cols, errors='ignore')
y_train = train_df_time['이탈_타겟']

X_test = test_df_time.drop(columns=drop_cols, errors='ignore')
y_test = test_df_time['이탈_타겟']

# 4. 모델 학습
model_final = XGBClassifier(n_estimators=100, learning_rate=0.05, max_depth=6, random_state=42)
model_final.fit(X_train, y_train)

# 5. 결과 확인
y_pred = model_final.predict(X_test)
print(f"--- {last_month} 데이터 기준 예측 결과 ---")
print(classification_report(y_test, y_pred))

--- 201812 데이터 기준 예측 결과 ---
              precision    recall  f1-score   support

           0       0.97      1.00      0.98     26615
           1       0.83      0.42      0.56      1378

    accuracy                           0.97     27993
   macro avg       0.90      0.71      0.77     27993
weighted avg       0.96      0.97      0.96     27993



In [14]:
import pandas as pd

# 1. 학습 데이터(Train)의 0과 1 분포 확인
train_counts = y_train.value_counts().sort_index()
train_pct = y_train.value_counts(normalize=True).sort_index() * 100

# 2. 테스트 데이터(Test)의 0과 1 분포 확인
test_counts = y_test.value_counts().sort_index()
test_pct = y_test.value_counts(normalize=True).sort_index() * 100

# 결과 출력
print("--- [Train 데이터 (과거~11월) 분포] ---")
for val in [0, 1]:
    print(f"타겟 {val}: {train_counts[val]:>6}건 ({train_pct[val]:.2f}%)")

print("\n--- [Test 데이터 (12월 당월) 분포] ---")
for val in [0, 1]:
    print(f"타겟 {val}: {test_counts[val]:>6}건 ({test_pct[val]:.2f}%)")

--- [Train 데이터 (과거~11월) 분포] ---
타겟 0: 134265건 (91.74%)
타겟 1:  12081건 (8.26%)

--- [Test 데이터 (12월 당월) 분포] ---
타겟 0:  26615건 (95.08%)
타겟 1:   1378건 (4.92%)


In [15]:
# 1. FT-Transformer 상위 변수 리스트 정의
top_features = [
    '여유_항공이용금액', '할부건수_무이자_12M_R12M', '이용가능여부_해외겸용_신용_본인', '이용카드수_체크',
    'IB상담건수_VOC불만_R6M', '이용금액_C페이_R6M', '이용금액_B페이_R3M', '연체일수_B2M',
    '할부금액_무이자_3M_R12M', '할부금액_3M_R12M', '자발한도감액금액_R12M', '이용후경과월_신용',
    '회원여부_이용가능_CA', '청구금액_R6M', '시장연체상환여부_R6M', '_1순위업종_이용금액',
    '_3순위납부업종', '인입일수_IB_R6M', '잔액_카드론_B4M', '할부금액_유이자_12M_R12M',
    '청구서발송여부_B0', '평잔_6M', '증감_RP건수_유선방송_전월', 'IB문의건수_CL_RV_R6M',
    '카드론이용월수_누적', '마일_적립포인트_R3M', '이용가맹점수', '이용금액_당사기타_R3M',
    '할부금액_14M_R12M', 'IB상담건수_VOC민원_R6M', '이용개월수_할부_무이자_R3M', '평잔_카드론_6M',
    '금액_할부전환_R6M', '이용금액_교육', '최종카드론이용경과월', '할부건수_유이자_14M_R12M',
    '이용건수_선결제_R6M', '연체원금_최근', '이용가능여부_해외겸용_본인', '증감율_이용금액_신판_분기',
    '쇼핑_슈퍼마켓_이용금액', '이용개월수_CA_R3M', '평잔_할부_해외_3M', '이용개월수_카드론_R3M',
    '증감_RP건수_렌탈_전월', '이용개월수_신판_R12M', '쇼핑_온라인_이용금액', '이용개월수_당사페이_R6M',
    '변동률_RV일시불평잔', '할부건수_부분_14M_R12M', '증감_RP건수_가스_전월', '이용금액_A페이_R6M',
    '이용카드수_신용체크', 'RP후경과월_아파트', '쇼핑_마트_이용금액', '잔액_카드론_B5M',
    '이용건수_할부_유이자_R3M', '이용건수_할부_무이자_R12M', 'RP후경과월_가스', '최종카드발급경과월',
    '증감율_이용건수_신판_분기', '이용개월수_일시불_R12M', '증감_RP건수_제휴사서비스직접판매_전월',
    '이용건수_선결제_R3M', '유효카드수_신용체크', '카드이용한도금액', '이용개월수_전체_R6M',
    '회원여부_연체', '이용개월수_C페이_R6M', '이용금액_CA_R12M', '이용금액_교통', '최초한도금액',
    '이용금액_체크_R6M', '강제한도감액금액_R12M', '평잔_CA_6M', '여유_전체이용금액',
    '승인거절건수_BL_R3M', '수신거부여부_DM', '이용개월수_선결제_R6M', '잔액_일시불_B1M'
]

# 2. 추가할 컬럼 리스트 정의
additional_cols = [
    '이용금액_신용_B0M', '이용금액_체크_B0M', '이용금액_신용_R3M', '이용금액_체크_R3M',
    '당월_총_이용금액', '직전_3M_평균_이용금액',
    '발급회원번호', '기준년월', '이탈_타겟'
]

# 3. 전체 컬럼 합치기 및 중복 제거
all_selected_cols = list(dict.fromkeys(top_features + additional_cols))

# 4. train_df에 실제로 있는 컬럼만 선택 (KeyError 방지)
final_cols = [col for col in all_selected_cols if col in train_df.columns]

# 5. 새로운 데이터프레임 생성
df_FTTransformer = train_df[final_cols].copy()

# 결과 확인
print(f"선택된 총 컬럼 수: {len(df_FTTransformer.columns)}")
display(df_FTTransformer.head())

선택된 총 컬럼 수: 89


,여유_항공이용금액,할부건수_무이자_12M_R12M,이용가능여부_해외겸용_신용_본인,이용카드수_체크,IB상담건수_VOC불만_R6M,이용금액_C페이_R6M,이용금액_B페이_R3M,연체일수_B2M,할부금액_무이자_3M_R12M,할부금액_3M_R12M,...,잔액_일시불_B1M,이용금액_신용_B0M,이용금액_체크_B0M,이용금액_신용_R3M,이용금액_체크_R3M,당월_총_이용금액,직전_3M_평균_이용금액,발급회원번호,기준년월,이탈_타겟
0,0,0,1,0,0,0,0,-99999999,1876273,2467580,...,459066,640723,0,1650182,0,640723,550060.666667,SYN_1000048,201807,0
1,0,0,1,0,0,0,0,-99999999,1251490,1968275,...,647858,553131,0,1402826,0,553131,467608.666667,SYN_1000048,201808,0
2,0,0,1,0,0,0,0,-99999999,890884,1566288,...,436254,593691,0,1787545,0,593691,595848.333333,SYN_1000048,201809,0
3,0,0,1,1,0,0,0,-99999999,891763,1601075,...,751680,548379,0,1695201,0,548379,565067.000000,SYN_1000048,201810,0
4,0,0,1,1,0,0,0,-99999999,792991,1380486,...,412978,592141,0,1734211,0,592141,578070.333333,SYN_1000048,201811,0


In [20]:
from lightgbm import LGBMClassifier

# 1. 클래스 불균형 처리를 위한 비율(ratio) 계산
# 타겟 변수의 0과 1의 분포를 확인하여 가중치 산출
count_0 = (y_train == 0).sum()
count_1 = (y_train == 1).sum()
ratio = count_0 / count_1

# 2. 데이터 타입 변환
# LightGBM의 범주형 데이터 처리를 위해 최적화
for col in X_train.select_dtypes(include=['object']).columns:
    X_train[col] = X_train[col].astype('category')
    X_test[col] = X_test[col].astype('category')

# 3. LightGBM 모델 생성 및 학습
# scale_pos_weight 파라미터에 위에서 계산한 ratio 적용
model_lgbm = LGBMClassifier(
    n_estimators=200,
    learning_rate=0.05,
    max_depth=7,
    random_state=42,
    scale_pos_weight=ratio,
    importance_type='gain'
)

print("LightGBM 학습을 시작합니다...")
model_lgbm.fit(X_train, y_train)
print("모델 학습이 완료되었습니다.")

LightGBM 학습을 시작합니다...
[LightGBM] [Info] Number of positive: 12081, number of negative: 134265
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.083451 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 18484
[LightGBM] [Info] Number of data points in the train set: 146346, number of used features: 86
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.082551 -> initscore=-2.408181
[LightGBM] [Info] Start training from score -2.408181
모델 학습이 완료되었습니다.


In [21]:
# 4. 결과 확인
y_pred = model_lgbm.predict(X_test)
print(f"\n--- {last_month} 데이터 기준 LightGBM 예측 결과 ---")
print(classification_report(y_test, y_pred))


--- 201812 데이터 기준 LightGBM 예측 결과 ---
              precision    recall  f1-score   support

           0       0.99      0.95      0.97     26615
           1       0.46      0.87      0.60      1378

    accuracy                           0.94     27993
   macro avg       0.73      0.91      0.79     27993
weighted avg       0.97      0.94      0.95     27993

